In [1]:
import numpy as np
import pandas as pd
from numpy.random import poisson
from scipy.stats import truncnorm

# Usando um parâmetro por jogador

In [2]:
def generate_games():
    games = [[chr(i + 65) for i in range(20)] for j in range(20)]
    for i in range(20):
        games[i].pop(i)
        
    return games

def generate_players(temps = 15):
    players = list(np.random.normal(70, 10, 220))
    players = [round(player, 1) if player < 100 else 100 for player in players]
    clubs = {}
    for year in range(temps):
        line_up = {}
        if type(players) != list:
            np.random.shuffle(list(players.flat))

        players = np.reshape(players, (20, 11))
        for club in range(20):
            line_up[chr(65 + club)] = players[club]

        clubs[year] = line_up
        
    return clubs

def find_median(array):
    arraycopy = array.copy()
    arraycopy.sort()
    if len(arraycopy) % 2 == 0:
        return (arraycopy[len(arraycopy) // 2 - 1] + arraycopy[len(arraycopy) // 2])/2
    else:
        return arraycopy[len(arraycopy) // 2]
    
def find_forces(clubs, year = 0):
    atk_forces = []
    def_forces = []
    for club in clubs[year]:
        atk_forces.append(np.sum(clubs[year][club]))
        def_forces.append(np.log(np.prod(clubs[year][club])))
            
    atk_med = find_median(atk_forces)
    def_med = find_median(def_forces)
    
    for i in range(len(atk_forces)):
        atk_forces[i] = atk_forces[i] / atk_med * 2
        def_forces[i] /= def_med
            
    return atk_forces, def_forces

def model(club1, club2, sims = 10000):
    '''
    receives 2 clubs (forces) and returns the most likely result, probability of each result and probability for the game
    '''
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson(club1[0] - club2[1])
        goals2 = poisson(club2[0] - club1[1])
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [3]:
def championship(clubs, sims = 10000, year = 0):
    atk_forces, def_forces = find_forces(clubs, year = year)
    games = generate_games()
    results = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            club1 = (atk_forces[home], def_forces[home])
            club2 = (atk_forces[away], def_forces[away])
            _, result, _ = model(club1, club2, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results

In [4]:
clubs = generate_players(temps = 1)
atk_forces, def_forces = find_forces(clubs)

# exemplo
club1 = (atk_forces[ord('A') - 65], def_forces[ord('A') - 65])
club2 = (atk_forces[ord('B') - 65], def_forces[ord('B') - 65])
results, result, probs = model(club1, club2, sims = 10000)

print('Forças do clube 1:', club1)
print('Forças do clube 2:', club2)
print()
print('Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):')
print(results)
print()
print('O resultado mais provável é {} x {}.'.format(result[0], result[1]))
print('Além disso, a probabilidade do mandante é {}, enquanto a do visitante é de {} e a de empate é {}.'.format(probs[0], probs[2], probs[1]))

Forças do clube 1: (1.904364276860852, 0.9885378438987577)
Forças do clube 2: (1.8352142996933913, 0.9763941263004995)

Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):
[[1699 1429  582  184   31    7]
 [1587 1321  557  175   30    8]
 [ 728  615  274   71   15    2]
 [ 220  181   79   24   11    1]
 [  51   49   21    8    1    0]
 [  17   11    8    2    0    1]]

O resultado mais provável é 0 x 0.
Além disso, a probabilidade do mandante é 0.3577, enquanto a do visitante é de 0.3103 e a de empate é 0.332.


In [5]:
df, results = championship(clubs, sims = 100)
df

,points,games,wins,draws,defeats,goals for,goals against,goal difference
P,83,38,24,11,3,31,10,21
G,67,38,17,16,5,34,22,12
C,55,38,13,16,9,23,19,4
I,55,38,13,16,9,22,18,4
L,55,38,12,19,7,25,20,5
R,54,38,12,18,8,20,15,5
K,54,38,11,21,6,20,15,5
S,53,38,12,17,9,23,20,3
T,53,38,12,17,9,20,17,3
E,51,38,12,15,11,15,14,1


In [6]:
results

[['A', 1, 0, 'B'],
 ['A', 0, 1, 'C'],
 ['A', 0, 1, 'D'],
 ['A', 0, 0, 'E'],
 ['A', 0, 1, 'F'],
 ['A', 1, 0, 'G'],
 ['A', 0, 1, 'H'],
 ['A', 1, 0, 'I'],
 ['A', 0, 1, 'J'],
 ['A', 1, 1, 'K'],
 ['A', 1, 1, 'L'],
 ['A', 1, 0, 'M'],
 ['A', 1, 1, 'N'],
 ['A', 1, 1, 'O'],
 ['A', 1, 1, 'P'],
 ['A', 0, 0, 'Q'],
 ['A', 0, 0, 'R'],
 ['A', 1, 1, 'S'],
 ['A', 0, 0, 'T'],
 ['B', 0, 0, 'A'],
 ['B', 0, 0, 'C'],
 ['B', 0, 1, 'D'],
 ['B', 0, 1, 'E'],
 ['B', 0, 1, 'F'],
 ['B', 1, 1, 'G'],
 ['B', 0, 0, 'H'],
 ['B', 0, 1, 'I'],
 ['B', 0, 1, 'J'],
 ['B', 0, 0, 'K'],
 ['B', 0, 0, 'L'],
 ['B', 0, 0, 'M'],
 ['B', 0, 1, 'N'],
 ['B', 0, 0, 'O'],
 ['B', 0, 1, 'P'],
 ['B', 1, 0, 'Q'],
 ['B', 1, 0, 'R'],
 ['B', 1, 1, 'S'],
 ['B', 1, 0, 'T'],
 ['C', 1, 0, 'A'],
 ['C', 0, 0, 'B'],
 ['C', 1, 1, 'D'],
 ['C', 1, 1, 'E'],
 ['C', 1, 0, 'F'],
 ['C', 0, 1, 'G'],
 ['C', 0, 0, 'H'],
 ['C', 0, 1, 'I'],
 ['C', 1, 0, 'J'],
 ['C', 0, 0, 'K'],
 ['C', 1, 1, 'L'],
 ['C', 1, 0, 'M'],
 ['C', 1, 1, 'N'],
 ['C', 1, 0, 'O'],
 ['C', 1, 1,

Perceba que fazendo

$força~de~ataque~do~time = \sum força~dos~jogadores \text{ e } força~de~defesa~do~time = \sum \log{força~dos~jogadores},$

um time com maior força de ataque terá maior força de defesa, o que não é compatível com a realidade.

# Usando dois parâmetros por jogador

Cada jogador tem dois parâmetros: ataque e defesa

In [7]:
def generate_players(clubs = 20, players_per_club = 23):
    '''
    creates players
    '''
    n_players = clubs * players_per_club
    players_atk = truncnorm.rvs(0, 10, loc = 2, scale = 0.5, size = n_players)
    players_def = truncnorm.rvs(0, 10, loc = 1, scale = 0.5, size = n_players)
    players = {}
    for i in range(n_players):
        players['P{:04d}'.format(i)] = [round(players_atk[i], 8) if players_atk[i] < 100 else 100,
                                        round(players_def[i], 8) if players_def[i] < 100 else 100]
        
    return players

def change_clubs(clubs, changes):
    '''
    change clubs
    '''
    clubs_f = {}
    changing = []
    change = 0
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        np.random.shuffle(club_players)
        for j in range(changes[i]):
            changing.append(club_players[j])

    np.random.shuffle(changing)
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        ind = []
        for player in club_players:
            if player in changing:
                ind.append(club_players.index(player))

        for j in range(len(ind)):
            club_players[ind[j]] = changing[change]
            change += 1

        clubs_f[chr(i + 65)] = club_players

    return clubs_f

def generate_clubs(players, years = 15, n_clubs = 20, players_per_club = 23):
    '''
    receives players and allocates them to clubs
    '''
    clubs = {}
    all_players = list(players)
    for year in range(years):
        line_up = {chr(65 + club) : {} for club in range(n_clubs)}
        clubs[year] = line_up
        for club in range(n_clubs):
            if year == 0:
                clubs[year][chr(65 + club)] = all_players[players_per_club * club:players_per_club * (club + 1)]

            else:
                changes = poisson(players_per_club/4, n_clubs)
                for i in range(n_clubs):
                    if changes[i] > players_per_club:
                        changes[i] = players_per_club

                    clubs[year] = change_clubs(clubs[year - 1], changes)
    return clubs

In [8]:
years = 15
n_clubs = 20
players_per_club = 23
players = generate_players()
clubs = generate_clubs(players)

In [9]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

def model(club1, club2, players, sims = 10000):
    '''
    receives 2 clubs (line_up) and returns the most likely result, probability of each result and probability for the game
    '''
    atk1, def1 = find_forces(club1, players)
    atk2, def2 = find_forces(club2, players)
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson(atk1 - def2)
        goals2 = poisson(atk2 - def1)
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [10]:
def championship(clubs, players, sims = 10000, year = 0):
    games = generate_games()
    results = []
    line_up = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            
            club1 = clubs[year][chr(home + 65)].copy()
            np.random.shuffle(club1)
            home_line_up = club1[:11]
            
            club2 = clubs[year][chr(away + 65)].copy()
            np.random.shuffle(club2)
            away_line_up = club2[:11]
            
            _, result, _ = model(home_line_up, away_line_up, players, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            line_up.append([home_line_up, away_line_up])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results, line_up

In [11]:
table, results, line_up = championship(clubs, players, sims = 100, year = 0)

In [12]:
table

,points,games,wins,draws,defeats,goals for,goals against,goal difference
P,67,38,18,13,7,80,62,18
I,56,38,14,14,10,66,61,5
S,56,38,13,17,8,77,67,10
C,55,38,15,10,13,66,67,-1
G,55,38,14,13,11,72,69,3
J,55,38,13,16,9,73,69,4
Q,54,38,12,18,8,73,68,5
R,52,38,14,10,14,61,65,-4
T,52,38,13,13,12,62,64,-2
O,51,38,11,18,9,68,65,3


In [13]:
results

[['A', 1, 1, 'B'],
 ['A', 2, 3, 'C'],
 ['A', 1, 2, 'D'],
 ['A', 2, 1, 'E'],
 ['A', 2, 1, 'F'],
 ['A', 3, 2, 'G'],
 ['A', 2, 3, 'H'],
 ['A', 2, 1, 'I'],
 ['A', 1, 3, 'J'],
 ['A', 2, 1, 'K'],
 ['A', 1, 1, 'L'],
 ['A', 1, 1, 'M'],
 ['A', 1, 2, 'N'],
 ['A', 2, 1, 'O'],
 ['A', 2, 2, 'P'],
 ['A', 1, 2, 'Q'],
 ['A', 1, 2, 'R'],
 ['A', 1, 3, 'S'],
 ['A', 1, 2, 'T'],
 ['B', 1, 3, 'A'],
 ['B', 1, 2, 'C'],
 ['B', 2, 1, 'D'],
 ['B', 2, 2, 'E'],
 ['B', 1, 2, 'F'],
 ['B', 3, 3, 'G'],
 ['B', 1, 3, 'H'],
 ['B', 2, 3, 'I'],
 ['B', 1, 1, 'J'],
 ['B', 3, 1, 'K'],
 ['B', 2, 1, 'L'],
 ['B', 1, 2, 'M'],
 ['B', 1, 1, 'N'],
 ['B', 1, 2, 'O'],
 ['B', 1, 2, 'P'],
 ['B', 2, 3, 'Q'],
 ['B', 1, 2, 'R'],
 ['B', 2, 2, 'S'],
 ['B', 1, 2, 'T'],
 ['C', 2, 1, 'A'],
 ['C', 1, 2, 'B'],
 ['C', 3, 2, 'D'],
 ['C', 3, 2, 'E'],
 ['C', 3, 2, 'F'],
 ['C', 1, 3, 'G'],
 ['C', 1, 2, 'H'],
 ['C', 1, 1, 'I'],
 ['C', 1, 2, 'J'],
 ['C', 1, 1, 'K'],
 ['C', 2, 1, 'L'],
 ['C', 2, 1, 'M'],
 ['C', 1, 4, 'N'],
 ['C', 2, 1, 'O'],
 ['C', 2, 3,

In [14]:
line_up

[[['P0007',
   'P0012',
   'P0015',
   'P0018',
   'P0017',
   'P0009',
   'P0022',
   'P0006',
   'P0011',
   'P0020',
   'P0019'],
  ['P0032',
   'P0023',
   'P0035',
   'P0039',
   'P0029',
   'P0027',
   'P0031',
   'P0043',
   'P0028',
   'P0041',
   'P0026']],
 [['P0018',
   'P0017',
   'P0006',
   'P0003',
   'P0001',
   'P0016',
   'P0004',
   'P0015',
   'P0011',
   'P0019',
   'P0020'],
  ['P0046',
   'P0047',
   'P0059',
   'P0066',
   'P0061',
   'P0065',
   'P0054',
   'P0057',
   'P0063',
   'P0050',
   'P0055']],
 [['P0016',
   'P0001',
   'P0005',
   'P0003',
   'P0009',
   'P0021',
   'P0018',
   'P0022',
   'P0007',
   'P0002',
   'P0020'],
  ['P0091',
   'P0073',
   'P0090',
   'P0086',
   'P0089',
   'P0082',
   'P0069',
   'P0077',
   'P0085',
   'P0083',
   'P0078']],
 [['P0018',
   'P0021',
   'P0014',
   'P0016',
   'P0020',
   'P0022',
   'P0008',
   'P0013',
   'P0003',
   'P0019',
   'P0006'],
  ['P0099',
   'P0104',
   'P0106',
   'P0096',
   'P0110',
   'P0